## Pyspark 전처리
- Web Ui : http://localhost:4040/

In [305]:
import pyspark as ps
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import regexp_extract, regexp_replace, col,concat,lit,split, udf

In [119]:
# 해당 부분 실행시 스파크 연결 되면서 Web Ui 접속 가능
spark = SparkSession.builder\
                    .master('local[*]')\
                    .appName('hello_world_app')\
                    .getOrCreate()

print(spark.version)

3.0.1


### 데이터 읽기

In [120]:
artist_df = spark.read.csv('artist.csv', header=True)

In [121]:
artist_df.show(5)

+----------------+-------------------------------+
|     artist_name|                    search_word|
+----------------+-------------------------------+
|TWICE (트와이스)|트와이스|트둥이|트 와이 스|T...|
|            MC몽|           원숭이|MC 몽|엠씨몽||
|      가비엔제이|                           null|
|             109|               일공구|10구|일09|
|     The Beatles|                 비틀즈|Beatles|
+----------------+-------------------------------+
only showing top 5 rows



In [122]:
# 칼럼별 읽기
artist_df_name = artist_df.select("artist_name")
artist_df_name.show()

+----------------+
|     artist_name|
+----------------+
|TWICE (트와이스)|
|            MC몽|
|      가비엔제이|
|             109|
|     The Beatles|
|              @@|
|      몬스타엑스|
|  태연 (TAEYEON)|
|     테스트 (abc|
+----------------+



###  데이터 출력

In [310]:
# coalesce(1): 분산된 상태로 저장하지 않기 위함
artist_df.coalesce(1).write.format("com.databricks.spark.csv").option("header", "false").save("output3.csv")

### 데이터 전처리

In [134]:
# 괄호 안에 있는 단어만 뽑아서 괄호안 단어만 수집
df_1 = artist_df_name.withColumn('artist_in', regexp_extract('artist_name', "\(.+?\)" , 0 ))
df_2 = df_1.withColumn('artist_in', regexp_replace('artist_in', "\(",""))
df_3 = df_2.withColumn('artist_in', regexp_replace('artist_in', "\)",""))
df_3.show()

+----------------+---------+
|     artist_name|artist_in|
+----------------+---------+
|TWICE (트와이스)| 트와이스|
|            MC몽|         |
|      가비엔제이|         |
|             109|         |
|     The Beatles|         |
|              @@|         |
|      몬스타엑스|         |
|  태연 (TAEYEON)|  TAEYEON|
|     테스트 (abc|         |
+----------------+---------+



In [146]:
# 한글과 띄어쓰기를 제외한 모든 글자
df_1 = artist_df_name.withColumn('artist_in', regexp_extract('artist_name', "[^ ㄱ-ㅣ가-힣]+", 0 ))
df_1.show()

+----------------+---------+
|     artist_name|artist_in|
+----------------+---------+
|TWICE (트와이스)|    TWICE|
|            MC몽|       MC|
|      가비엔제이|         |
|             109|      109|
|     The Beatles|      The|
|              @@|       @@|
|      몬스타엑스|         |
|  태연 (TAEYEON)|(TAEYEON)|
|     테스트 (abc|     (abc|
+----------------+---------+



In [158]:
# 한글이 있다면 한글만 추출
df_1 = artist_df_name.withColumn('artist_in', regexp_extract('artist_name', "[가-힣]+", 0 ))
df_1.show()

+----------------+----------+
|     artist_name| artist_in|
+----------------+----------+
|TWICE (트와이스)|  트와이스|
|            MC몽|        몽|
|      가비엔제이|가비엔제이|
|             109|          |
|     The Beatles|          |
|              @@|          |
|      몬스타엑스|몬스타엑스|
|  태연 (TAEYEON)|      태연|
|     테스트 (abc|    테스트|
+----------------+----------+



In [157]:
# 한글로만 구성 (\\s 는 안에 띄어쓰기가 있어도 추출)
df_1 = artist_df_name.withColumn('artist_in', regexp_extract('artist_name', "^[ㄱ-ㅎ가-힣\\s]*$", 0 ))
df_1.show()

+----------------+----------+
|     artist_name| artist_in|
+----------------+----------+
|TWICE (트와이스)|          |
|            MC몽|          |
|      가비엔제이|가비엔제이|
|             109|          |
|     The Beatles|          |
|              @@|          |
|      몬스타엑스|몬스타엑스|
|  태연 (TAEYEON)|          |
|     테스트 (abc|          |
+----------------+----------+



In [156]:
# 숫자로만 구성  (\\s 는 안에 띄어쓰기가 있어도 추출)
df_1 = artist_df_name.withColumn('artist_in', regexp_extract('artist_name', "^[0-9\\s]*$", 0 ))
df_1.show()

+----------------+---------+
|     artist_name|artist_in|
+----------------+---------+
|TWICE (트와이스)|         |
|            MC몽|         |
|      가비엔제이|         |
|             109|      109|
|     The Beatles|         |
|              @@|         |
|      몬스타엑스|         |
|  태연 (TAEYEON)|         |
|     테스트 (abc|         |
+----------------+---------+



In [151]:
# 영어로만 구성  (\\s 는 안에 띄어쓰기가 있어도 추출)
df_1 = artist_df_name.withColumn('artist_in', regexp_extract('artist_name', "^[a-zA-Z\\s]*$", 0 ))
df_1.show()

+----------------+-----------+
|     artist_name|  artist_in|
+----------------+-----------+
|TWICE (트와이스)|           |
|            MC몽|           |
|      가비엔제이|           |
|             109|           |
|     The Beatles|The Beatles|
|              @@|           |
|      몬스타엑스|           |
|  태연 (TAEYEON)|           |
|     테스트 (abc|           |
+----------------+-----------+



In [159]:
# 특수문자로만 구성  (\W 는 문자+숫자가 아닌 특수문자+공백 추출임)
df_1 = artist_df_name.withColumn('artist_in', regexp_extract('artist_name', "^[\W]*$", 0 ))
df_1.show()

+----------------+----------+
|     artist_name| artist_in|
+----------------+----------+
|TWICE (트와이스)|          |
|            MC몽|          |
|      가비엔제이|가비엔제이|
|             109|          |
|     The Beatles|          |
|              @@|        @@|
|      몬스타엑스|몬스타엑스|
|  태연 (TAEYEON)|          |
|     테스트 (abc|          |
+----------------+----------+



In [162]:
# 영어 or 한글 or 영어+한글로 되어있는 글자만 추출(영+한+특수문자는 안됨.)
df_1 = artist_df_name.withColumn('artist_in', regexp_extract('artist_name', "^[가-힣ㄱ-ㅎA-Za-z]*$", 0 ))
df_1.show()

+----------------+----------+
|     artist_name| artist_in|
+----------------+----------+
|TWICE (트와이스)|          |
|            MC몽|      MC몽|
|      가비엔제이|가비엔제이|
|             109|          |
|     The Beatles|          |
|              @@|          |
|      몬스타엑스|몬스타엑스|
|  태연 (TAEYEON)|          |
|     테스트 (abc|          |
+----------------+----------+



In [166]:
# 영어+한글로 되어있는 글자만 추출(영+한+특수문자는 안됨.)
df_1 = artist_df_name.withColumn('artist_in', regexp_extract('artist_name', "^(^[가-힣ㄱ-ㅎA-Za-z]*$)", 0 ))
df_1.show()

+----------------+----------+
|     artist_name| artist_in|
+----------------+----------+
|TWICE (트와이스)|          |
|            MC몽|      MC몽|
|      가비엔제이|가비엔제이|
|             109|          |
|     The Beatles|          |
|              @@|          |
|      몬스타엑스|몬스타엑스|
|  태연 (TAEYEON)|          |
|     테스트 (abc|          |
+----------------+----------+



In [171]:
# 한글이 안들어 있는 데이터
df_1 = artist_df_name.withColumn('artist_in', regexp_extract('artist_name', "^[^가-힣]*$", 0 ))
df_1.show()

+----------------+-----------+
|     artist_name|  artist_in|
+----------------+-----------+
|TWICE (트와이스)|           |
|            MC몽|           |
|      가비엔제이|           |
|             109|        109|
|     The Beatles|The Beatles|
|              @@|         @@|
|      몬스타엑스|           |
|  태연 (TAEYEON)|           |
|     테스트 (abc|           |
+----------------+-----------+



In [184]:
# 영어만으로 이뤄진거 제거
df_1 = artist_df_name.withColumn('artist_in', regexp_extract('artist_name', "^[a-zA-Z\\s]*$", 0 ))
df_1.where(df_1.artist_in == '').show()

+----------------+---------+
|     artist_name|artist_in|
+----------------+---------+
|TWICE (트와이스)|         |
|            MC몽|         |
|      가비엔제이|         |
|             109|         |
|              @@|         |
|      몬스타엑스|         |
|  태연 (TAEYEON)|         |
|     테스트 (abc|         |
+----------------+---------+



In [208]:
# 두개의 칼럼 하나의 열로 합치기(빈값 제거, distinct로 중복 제거)
unionDF = df_1.select("artist_name").where(df_1.artist_name != '').union(artist_df_name).where(df_1.artist_name != '').distinct()

In [209]:
unionDF.show()

+----------------+
|     artist_name|
+----------------+
|      몬스타엑스|
|TWICE (트와이스)|
|      가비엔제이|
|              @@|
|            MC몽|
|     테스트 (abc|
|  태연 (TAEYEON)|
|     The Beatles|
|             109|
+----------------+



In [211]:
# 데이터 출력
unionDF.coalesce(1).write.format("com.databricks.spark.csv").option("header", "false").save("artist_all.csv")

In [213]:
# 공백 제거
df_3 = df_1.withColumn('artist_name', regexp_replace('artist_name', " ",""))
df_3.show()

+---------------+-----------+
|    artist_name|  artist_in|
+---------------+-----------+
|TWICE(트와이스)|           |
|           MC몽|           |
|     가비엔제이|           |
|            109|           |
|     TheBeatles|The Beatles|
|             @@|           |
|     몬스타엑스|           |
|  태연(TAEYEON)|           |
|     테스트(abc|           |
+---------------+-----------+



### 검색어 데이터

In [297]:
# 검색어 데이터 읽기 (검색어 데이터가 있는 경우 에만 데이터 만들기)
artist_df_search = artist_df.where((artist_df.search_word != '') & (artist_df.search_word.isNotNull()))
artist_df_search.show()

+----------------+-------------------------------+
|     artist_name|                    search_word|
+----------------+-------------------------------+
|TWICE (트와이스)|트와이스|트둥이|트 와이 스|T...|
|            MC몽|           원숭이|MC 몽|엠씨몽||
|             109|               일공구|10구|일09|
|     The Beatles|                 비틀즈|Beatles|
|              @@|                     골뱅이|@두|
|      몬스타엑스| 몬스타X|몬스타 엑스|몬스터엑스|
|  태연 (TAEYEON)|                   태연|TAEYEON|
|     테스트 (abc|                     테스트|abc|
+----------------+-------------------------------+



In [298]:
# 공백제거
artist_df_search = artist_df_search.withColumn('artist_name', regexp_replace('artist_name', " ",""))
artist_df_search = artist_df_search.withColumn('search_word', regexp_replace('search_word', " ",""))
artist_df_search.show()

+---------------+------------------------------+
|    artist_name|                   search_word|
+---------------+------------------------------+
|TWICE(트와이스)|트와이스|트둥이|트와이스|TWICE|
|           MC몽|           원숭이|MC몽|엠씨몽||
|            109|              일공구|10구|일09|
|     TheBeatles|                비틀즈|Beatles|
|             @@|                    골뱅이|@두|
|     몬스타엑스| 몬스타X|몬스타엑스|몬스터엑스|
|  태연(TAEYEON)|                  태연|TAEYEON|
|     테스트(abc|                    테스트|abc|
+---------------+------------------------------+



In [299]:
# 괄호가 있는경우 ()안에 텍스트만 추출
artist_df_search = artist_df_search.withColumn('artist_in', regexp_extract('artist_name', "\(.+?\)" , 0 ))
artist_df_search = artist_df_search.withColumn('artist_in', regexp_replace('artist_in', "\(",""))
artist_df_search = artist_df_search.withColumn('artist_in', regexp_replace('artist_in', "\)",""))
artist_df_search.show()

+---------------+------------------------------+---------+
|    artist_name|                   search_word|artist_in|
+---------------+------------------------------+---------+
|TWICE(트와이스)|트와이스|트둥이|트와이스|TWICE| 트와이스|
|           MC몽|           원숭이|MC몽|엠씨몽||         |
|            109|              일공구|10구|일09|         |
|     TheBeatles|                비틀즈|Beatles|         |
|             @@|                    골뱅이|@두|         |
|     몬스타엑스| 몬스타X|몬스타엑스|몬스터엑스|         |
|  태연(TAEYEON)|                  태연|TAEYEON|  TAEYEON|
|     테스트(abc|                    테스트|abc|         |
+---------------+------------------------------+---------+



In [300]:
# 한글이 포함된 글자만 추출
artist_df_search = artist_df_search.withColumn('artist_kor', regexp_extract('artist_name', "[ㄱ-ㅎ가-힣]+", 0 ))
artist_df_search.show()

+---------------+------------------------------+---------+----------+
|    artist_name|                   search_word|artist_in|artist_kor|
+---------------+------------------------------+---------+----------+
|TWICE(트와이스)|트와이스|트둥이|트와이스|TWICE| 트와이스|  트와이스|
|           MC몽|           원숭이|MC몽|엠씨몽||         |        몽|
|            109|              일공구|10구|일09|         |          |
|     TheBeatles|                비틀즈|Beatles|         |          |
|             @@|                    골뱅이|@두|         |          |
|     몬스타엑스| 몬스타X|몬스타엑스|몬스터엑스|         |몬스타엑스|
|  태연(TAEYEON)|                  태연|TAEYEON|  TAEYEON|      태연|
|     테스트(abc|                    테스트|abc|         |    테스트|
+---------------+------------------------------+---------+----------+



In [301]:
# 구분자 칼럼 추가 
artist_df_search = artist_df_search.withColumn('sep',lit("|"))
artist_df_search.show()

+---------------+------------------------------+---------+----------+---+
|    artist_name|                   search_word|artist_in|artist_kor|sep|
+---------------+------------------------------+---------+----------+---+
|TWICE(트와이스)|트와이스|트둥이|트와이스|TWICE| 트와이스|  트와이스|  ||
|           MC몽|           원숭이|MC몽|엠씨몽||         |        몽|  ||
|            109|              일공구|10구|일09|         |          |  ||
|     TheBeatles|                비틀즈|Beatles|         |          |  ||
|             @@|                    골뱅이|@두|         |          |  ||
|     몬스타엑스| 몬스타X|몬스타엑스|몬스터엑스|         |몬스타엑스|  ||
|  태연(TAEYEON)|                  태연|TAEYEON|  TAEYEON|      태연|  ||
|     테스트(abc|                    테스트|abc|         |    테스트|  ||
+---------------+------------------------------+---------+----------+---+



In [302]:
# 뒤에 아티스트명 붙이기
artist_df_search = artist_df_search.withColumn('artist_concat', concat("artist_name","sep","search_word"))
artist_df_search.select("artist_concat").show()

+---------------------------------+
|                    artist_concat|
+---------------------------------+
|     TWICE(트와이스)|트와이스|...|
|         MC몽|원숭이|MC몽|엠씨몽||
|             109|일공구|10구|일09|
|          TheBeatles|비틀즈|Be...|
|                    @@|골뱅이|@두|
|몬스타엑스|몬스타X|몬스타엑스|...|
|         태연(TAEYEON)|태연|TA...|
|            테스트(abc|테스트|abc|
+---------------------------------+



In [307]:
# 행기준 중복 데이터 제거

In [309]:
# 검색어 데이터 '|' 기준 ,로 변경하여 붙이기
artist_df_search = artist_df_search.withColumn('artist_concat', regexp_replace('artist_concat', "\|","\,"))
artist_df_search.select("artist_concat").show()

+---------------------------------+
|                    artist_concat|
+---------------------------------+
|     TWICE(트와이스),트와이스,...|
|         MC몽,원숭이,MC몽,엠씨몽,|
|             109,일공구,10구,일09|
|          TheBeatles,비틀즈,Be...|
|                    @@,골뱅이,@두|
|몬스타엑스,몬스타X,몬스타엑스,...|
|         태연(TAEYEON),태연,TA...|
|            테스트(abc,테스트,abc|
+---------------------------------+



In [214]:
# 파일로 떨구기